In [1]:
import os
import sys

import re
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.collections import PatchCollection
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

import pandas as pd
import seaborn as sns
import json
from natsort import natsorted

In [2]:
otu_table = pd.read_csv("C:/Users/edwa0468/Downloads/p_otu.tsv", delimiter="\t")
sample_data = pd.read_csv("C:/Users/edwa0468/Downloads/p_sample.tsv", delimiter="\t")
tax_table = pd.read_csv("C:/Users/edwa0468/Downloads/p_tax.tsv", delimiter="\t")

otu_table

FileNotFoundError: [Errno 2] No such file or directory: 'C:/Users/edwa0468/Downloads/p_otu.tsv'

In [ ]:

sample_data

In [ ]:

tax_table